# Standard evaluators and target functions.
In this notebook we will demonstrate how to use the target functions with the standard evaluators.

In [ ]:
import inspect
import json
import pandas as pd
import os

from pprint import pprint
from promptflow.core import AzureOpenAIModelConfiguration
from promptflow.evals.evaluate import evaluate
from promptflow.evals.evaluators import F1ScoreEvaluator, RelevanceEvaluator

## Target function
We will use a simple `Ask Wiki` application to get answers to questions from wikipedia. We will use `evaluate` API to evaluate `Ask Wiki` application

`Ask Wiki` needs following environment variables to be set

- AZURE_OPENAI_API_KEY
- AZURE_OPENAI_API_VERSION
- AZURE_OPENAI_DEPLOYMENT
- AZURE_OPENAI_ENDPOINT

In [ ]:
# Use the following code to set the environment variables if not already set. If set, you can skip this step.

os.environ["AZURE_OPENAI_API_KEY"] = "<api_key>"
os.environ["AZURE_OPENAI_API_VERSION"] = "<api_version>"
os.environ["AZURE_OPENAI_DEPLOYMENT"] = "<deployment>"
os.environ["AZURE_OPENAI_ENDPOINT"] = "<endpoint>"

In [ ]:
from askwiki.askwiki import ask_wiki

ask_wiki("What is the capital of India?")

## Data
Reading existing dataset which has bunch of questions we can Ask Wiki

In [ ]:
df = pd.read_json('data/data.jsonl', lines=True)
print(df.head())

## Configuration
To use Relevance Evaluator, we will Azure Open AI model details that can be passed as model config.


In [ ]:
configuration = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"] ,
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT"],
)

## Running Relevance Evaluator to understand its input and output

In [ ]:
relevance_evaluator = RelevanceEvaluator(model_config=configuration)

relevance_evaluator(question="What is the capital of India?", answer="New Delhi is Capital of India" ,context="India is a country in South Asia.")

## Running custom evaluator **BlockList**

In [ ]:
# Note: The following evaluator is a defined in directory evaluators/blocklist

from evaluators.blocklist.blocklist import BlocklistEvaluator

blocklist_evaluator = BlocklistEvaluator(blocklist=["bad, worst, terrible"])

blocklist_evaluator(answer="New Delhi is Capital of India")

## Run the evaluation

In [ ]:
results = evaluate(
    data="data/data.jsonl",
    target=ask_wiki,
    evaluators={
        "blocklist": blocklist_evaluator,
        "relevance": relevance_evaluator,
    },
)

View the results

In [ ]:
pprint(results)

In [ ]:
pd.DataFrame(results["rows"])